In [1]:
import pandas as pd
import os
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'regresiPossionYTRewind'
collection_label = 'labelingData2018'
collection_name = 'fullLabeledData2018'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]
labeling_data = db[collection_label]
full_labeled_data = db[collection_name]

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_data.find())
unlabeled_data = list(full_labeled_data.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Menggunakan SMOTE untuk melakukan oversampling pada data berlabel
smote = SMOTE(random_state=42)
X_labeled_tfidf_res, y_labeled_res = smote.fit_resample(X_labeled_tfidf, y_labeled)

# Melatih model menggunakan data berlabel yang telah di-oversample
model = MultinomialNB()
model.fit(X_labeled_tfidf_res, y_labeled_res)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeled_data.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print(f"Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [4]:
# # Mengambil data dari koleksi MongoDB
# labeled_data = list(labeling_data.find())
# unlabeled_data = list(full_labeled_data.find())

# # Konversi data menjadi dataframe
# df_labeled = pd.DataFrame(labeled_data)
# df_unlabeled = pd.DataFrame(unlabeled_data)

# # Memisahkan fitur dan label dari data yang berlabel
# X_labeled = df_labeled['textOriginal']
# y_labeled = df_labeled['label']

# # # Evaluasi model dengan membagi data berlabel menjadi train dan test set
# # X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)

# # Train Data
# # Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
# vectorizer = TfidfVectorizer()
# X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# # Melatih model menggunakan Naive Bayes
# model = MultinomialNB()
# model.fit(X_labeled_tfidf, y_labeled)

# # Memproses data yang tidak berlabel
# X_unlabeled = df_unlabeled['textOriginal']
# X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# # Memprediksi label untuk data yang tidak berlabel
# y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# # Menambahkan prediksi ke dalam dataframe data tidak berlabel
# df_unlabeled['predicted_label'] = y_unlabeled_pred

# # Mengupdate koleksi MongoDB dengan prediksi label
# for index, row in df_unlabeled.iterrows():
#     full_labeled_data.update_one(
#         {"_id": row["_id"]},
#         {"$set": {"predicted_label": row["predicted_label"]}}
#     )

# print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")


#### Accuracy Test

In [5]:
# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_data.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Evaluasi model dengan membagi data berlabel menjadi train dan test set
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Menggunakan SMOTE untuk melakukan oversampling pada data latih
smote = SMOTE(random_state=42)
X_train_tfidf_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Melatih model menggunakan data train yang telah di-oversample
model = MultinomialNB()
model.fit(X_train_tfidf_res, y_train_res)

# Memprediksi data test
y_test_pred = model.predict(X_test_tfidf)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Akurasi model: {accuracy}")

# Mencetak classification report
report = classification_report(y_test, y_test_pred)
print(f"Classification Report:\n{report}")

Akurasi model: 0.625
Classification Report:
              precision    recall  f1-score   support

     negatif       0.44      0.69      0.54        81
      netral       0.79      0.51      0.62       196
     positif       0.64      0.77      0.70       123

    accuracy                           0.62       400
   macro avg       0.62      0.66      0.62       400
weighted avg       0.67      0.62      0.63       400



In [6]:
# Nama database dan koleksi
db_name = 'cobaAnalisisYTRewindIndonesia'
collection_name = 'labelingData2018'

# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]
collection = db[collection_name]

# Query agregasi untuk menghitung jumlah setiap kelas berdasarkan predicted_label
pipeline = [
    {"$group": {"_id": "$label", "count": {"$sum": 1}}},
    {"$match": {"_id": {"$in": ["positif", "negatif", "netral"]}}}
]

# Menjalankan query agregasi
results = collection.aggregate(pipeline)

# Menampilkan hasil
for result in results:
    print(f"Kelas: {result['_id']}, Jumlah: {result['count']}")

# Menutup koneksi
client.close()

Kelas: netral, Jumlah: 925
Kelas: positif, Jumlah: 603
Kelas: negatif, Jumlah: 472


In [7]:
# Nama database dan koleksi
db_name = 'cobaAnalisisYTRewindIndonesia'
collection_name = 'fullLabeledData2018'

# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]
collection = db[collection_name]

# Query agregasi untuk menghitung jumlah setiap kelas berdasarkan predicted_label
pipeline = [
    {"$group": {"_id": "$predicted_label", "count": {"$sum": 1}}},
    {"$match": {"_id": {"$in": ["positif", "negatif", "netral"]}}}
]

# Menjalankan query agregasi
results = collection.aggregate(pipeline)

# Menampilkan hasil
for result in results:
    print(f"Kelas: {result['_id']}, Jumlah: {result['count']}")

# Menutup koneksi
client.close()

Kelas: netral, Jumlah: 3491
Kelas: negatif, Jumlah: 3208
Kelas: positif, Jumlah: 3301


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Nama database dan collection
db_name = 'cobaAnalisisYTRewindIndonesia'
collection_name = 'fullLabeledData2018'

# Memuat value dari file .env
load_dotenv()

local_url = os.getenv('URL_LOCAL')

# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]
collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
data = list(collection.find())

# Konversi data menjadi dataframe
df = pd.DataFrame(data)

# Memisahkan komentar positif dan negatif
df_positive = df[df['predicted_label'] == 'positif']
df_negative = df[df['predicted_label'] == 'negatif']

def lda_topic_modeling(texts, n_topics=5, n_words=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)
    words = vectorizer.get_feature_names_out()
    topics = []
    for idx, topic in enumerate(lda.components_):
        topics.append([words[i] for i in topic.argsort()[:-n_words - 1:-1]])
    return topics

# Ekstraksi topik dari komentar positif
positive_topics = lda_topic_modeling(df_positive['textOriginal'], n_topics=1, n_words=20)
print("Topik dalam komentar positif:")
for i, topic in enumerate(positive_topics):
    print(f"Topik {i + 1}: {', '.join(topic)}")

# Ekstraksi topik dari komentar negatif
negative_topics = lda_topic_modeling(df_negative['textOriginal'], n_topics=1, n_words=20)
print("\nTopik dalam komentar negatif:")
for i, topic in enumerate(negative_topics):
    print(f"Topik {i + 1}: {', '.join(topic)}")


Topik dalam komentar positif:
Topik 1: keren, rewind, indonesia, bagus, youtube, lebih, banget, nya, mantap, the, god, sih, best, yt, job, ytr, buat, sama, lah, merinding

Topik dalam komentar negatif:
Topik 1: dislike, nya, ga, kurang, bowo, rewind, bagus, banyak, aja, kok, gk, sama, youtube, sih, youtuber, trending, semua, lebih, gua, yang


In [9]:
# # Evaluasi model dengan membagi data berlabel menjadi train dan test set
# X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)

# # Transformasi data train dan test menggunakan TF-IDF
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # Melatih model menggunakan data train
# model.fit(X_train_tfidf, y_train)

# # Memprediksi data test
# y_test_pred = model.predict(X_test_tfidf)

# # Menghitung akurasi
# accuracy = accuracy_score(y_test, y_test_pred)
# print(f"Akurasi model: {accuracy}")

# # Mencetak classification report
# report = classification_report(y_test, y_test_pred)
# print(f"Classification Report:\n{report}")

In [10]:
# # Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
# if 'label' in df_unlabeled.columns:
#     print("\nEvaluasi kinerja model:")
#     print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
#     print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
# else:
#     print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")